In [5]:
import torch as th
import dgl
import os
import torch.nn.functional as F
from models import GraphSageModel, GraphConvModel, GraphAttnModel


In [6]:
output_dir = "./output"
model_name = "dgl_model-057137.pth"
gnn_model = "graphsage"

model_path = os.path.join(output_dir, model_name)

in_feat = 300
hidden_dim = 64 
n_layers = 2
n_classes = 23

if gnn_model == 'graphsage':
    model = GraphSageModel(in_feat, hidden_dim, n_layers, n_classes)
elif gnn_model == 'graphconv':
    model = GraphConvModel(in_feat, hidden_dim, n_layers, n_classes,
                           norm='both', activation=F.relu, dropout=0)
elif gnn_model == 'graphattn':
    model = GraphAttnModel(in_feat, hidden_dim, n_layers, n_classes,
                           heads=([5] * n_layers), activation=F.relu, feat_drop=0, attn_drop=0)
else:
    raise NotImplementedError('So far, only support three algorithms: GraphSage, GraphConv, and GraphAttn')

In [9]:
?th.load

Signature:
th.load(
    f,
    map_location=None,
    pickle_module=<module 'pickle' from '/usr/lib/python3.8/pickle.py'>,
    **pickle_load_args,
)
Docstring:
Loads an object saved with :func:`torch.save` from a file.

:func:`torch.load` uses Python's unpickling facilities but treats storages,
which underlie tensors, specially. They are first deserialized on the
CPU and are then moved to the device they were saved from. If this fails
(e.g. because the run time system doesn't have certain devices), an exception
is raised. However, storages can be dynamically remapped to an alternative
set of devices using the :attr:`map_location` argument.

If :attr:`map_location` is a callable, it will be called once for each serialized
storage with two arguments: storage and location. The storage argument
will be the initial deserialization of the storage, residing on the CPU.
Each serialized storage has a location tag associated with it which
identifies the device it was saved from, and this tag is 

In [15]:
params = th.load(model_path)
model.load_state_dict(params)

<All keys matched successfully>

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_backward_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persistent_buffers_set',
 '_parameters',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_version',
 'activation',
 'add_modu